# 확장기 변수를 활용해 전체 데이터셋, 전환기 데이터셋 테스트

In [27]:
import pandas as pd
from scipy.stats.mstats import winsorize
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
rc('font', family='Malgun Gothic')
plt.rcParams['axes.unicode_minus'] = False
# from matplotlib import rc
# rc('font', family='AppleGothic')
# plt.rcParams['axes.unicode_minus'] = False

import warnings
warnings.filterwarnings(action='ignore')

In [28]:
expansion_normal_win = pd.read_csv('../Data/WIN/expansion_normal_win.csv',encoding='CP949')
recession_normal_win = pd.read_csv('../Data/WIN/recession_normal_win.csv',encoding='CP949')
transition_normal_win =  pd.read_csv('../Data/WIN/transition_normal_win.csv',encoding='CP949')
expansion_default_win =  pd.read_csv('../Data/WIN/expansion_default_win.csv',encoding='CP949')
recession_default_win = pd.read_csv('../Data/WIN/recession_default_win.csv',encoding='CP949')
transition_default_win =  pd.read_csv('../Data/WIN/transition_default_win.csv',encoding='CP949')

In [29]:
transition_normal = pd.read_csv('../Data/WIN/transition_normal_win.csv',encoding='cp949')
transition_default = pd.read_csv('../Data/WIN/transition_default_win.csv',encoding='cp949')
transition = pd.concat([transition_normal,transition_default],axis=0)

transition_00 = transition[transition['회계년도']==2000]
transition_01 = transition[transition['회계년도']==2001]
transition_05 = transition[transition['회계년도']==2005]
transition_11 = transition[transition['회계년도']==2011]
transition_13 = transition[transition['회계년도']==2013]
transition_17 = transition[transition['회계년도']==2017]

In [30]:
expansion = pd.concat([expansion_normal_win, expansion_default_win], axis=0)
recession = pd.concat([recession_normal_win,recession_default_win], axis=0)
transition = pd.concat([transition_normal_win,transition_default_win],axis=0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler() 
recession.iloc[:,3:-1]=scaler.fit_transform(recession.iloc[:,3:-1])

# normal = recession[recession['부도']==0]
# default = recession[recession['부도']==1]

In [31]:
# Levene's test 
from scipy import stats
from scipy.stats import levene
levene_list = []
expansion_list_pick = []
expansion_list_drop = []

for col in expansion_normal_win.iloc[:,3:-1].columns: 
    a = levene(expansion_normal_win.loc[:,col],expansion_default_win.loc[:,col])
    levene_list.append(a)

expansion_df = pd.DataFrame(levene_list, index= expansion_normal_win.iloc[:,3:-1].columns)
s_t_test = expansion_df[expansion_df['pvalue']>0.05] #H0 귀무가설을 채택 => 등분산성
w_t_test = expansion_df[expansion_df['pvalue']<=0.05] #H1 귀무가설 기각 => 이분산성 

for col in s_t_test.loc[:,'pvalue'].index:
    s_t, s_p = stats.ttest_ind(expansion_normal_win.loc[:,col], expansion_default_win.loc[:,col], equal_var= True)
                                                                                    #equal_var= False : 이분산 
                                                                                    #equal_var= True : 등분산
    if s_p <= 0.05 :
        print('\n')
        print("student's t test Result\n ------------------------")
        print(f' {col} 등분산 t값은 {s_t:.3f} 이다. ')
        print(f' {col} 등분산 p값은 {s_p:.3f} 이다. ')
        expansion_list_pick.append(col)
    else:
        expansion_list_drop.append(col)

from scipy import stats
for col in w_t_test.loc[:,'pvalue'].index:
    w_t, w_p = stats.ttest_ind(expansion_normal_win.loc[:,col], expansion_default_win.loc[:,col], equal_var= False)
    if w_p <= 0.05 :
        print("welch's t test Result\n ------------------------")
        print(f' {col} 이분산 t값은 {w_t:.3f} 이다. ')
        print(f' {col} 이분산 p값은 {w_p:.3f} 이다. ')  
        expansion_list_pick.append(col)
    else:
        expansion_list_drop.append(col)

print(expansion_list_pick)



student's t test Result
 ------------------------
 매출채권회전률 등분산 t값은 2.114 이다. 
 매출채권회전률 등분산 p값은 0.035 이다. 


student's t test Result
 ------------------------
 매입채무회전률 등분산 t값은 2.515 이다. 
 매입채무회전률 등분산 p값은 0.012 이다. 
welch's t test Result
 ------------------------
 유동비율 이분산 t값은 8.434 이다. 
 유동비율 이분산 p값은 0.000 이다. 
welch's t test Result
 ------------------------
 차입금의존도 이분산 t값은 -7.992 이다. 
 차입금의존도 이분산 p값은 0.000 이다. 
welch's t test Result
 ------------------------
 매출액증가율 이분산 t값은 2.796 이다. 
 매출액증가율 이분산 p값은 0.007 이다. 
welch's t test Result
 ------------------------
 매출액총이익률 이분산 t값은 5.651 이다. 
 매출액총이익률 이분산 p값은 0.000 이다. 
welch's t test Result
 ------------------------
 매출액영업이익률 이분산 t값은 7.210 이다. 
 매출액영업이익률 이분산 p값은 0.000 이다. 
welch's t test Result
 ------------------------
 매출액순이익률 이분산 t값은 6.101 이다. 
 매출액순이익률 이분산 p값은 0.000 이다. 
welch's t test Result
 ------------------------
 총자본영업이익률 이분산 t값은 10.142 이다. 
 총자본영업이익률 이분산 p값은 0.000 이다. 
welch's t test Result
 ------------------------
 자기자본순이익률 이분

In [32]:
expansion_list_pick

['매출채권회전률',
 '매입채무회전률',
 '유동비율',
 '차입금의존도',
 '매출액증가율',
 '매출액총이익률',
 '매출액영업이익률',
 '매출액순이익률',
 '총자본영업이익률',
 '자기자본순이익률',
 '당좌비율',
 '부채비율',
 '총자본회전률',
 'RETA',
 'TLTA',
 '이자보상배율',
 'vol유동비율',
 'vol매출액총이익률',
 'vol매출액영업이익률',
 'vol총자본영업이익률',
 'vol당좌비율',
 'vol이자보상배율']

In [33]:
pick_lst =['회사명', '거래소코드', '회계년도','매출채권회전률',
 '매입채무회전률',
 '유동비율',
 '차입금의존도',
 '매출액증가율',
 '매출액총이익률',
 '매출액영업이익률',
 '매출액순이익률',
 '총자본영업이익률',
 '자기자본순이익률',
 '당좌비율',
 '부채비율',
 '총자본회전률',
 'RETA',
 'TLTA',
 '이자보상배율',
 'vol유동비율',
 'vol매출액총이익률',
 'vol매출액영업이익률',
 'vol총자본영업이익률',
 'vol당좌비율',
 'vol이자보상배율','부도']

In [34]:
expansion = expansion[pick_lst]
expansion

,회사명,거래소코드,회계년도,매출채권회전률,매입채무회전률,유동비율,차입금의존도,매출액증가율,매출액총이익률,매출액영업이익률,...,RETA,TLTA,이자보상배율,vol유동비율,vol매출액총이익률,vol매출액영업이익률,vol총자본영업이익률,vol당좌비율,vol이자보상배율,부도
0,(주)CMG제약,58820,2002,18.91,58.88,114.21,53.16,66.81,0.00,-14.46,...,-0.139228,0.575696,-9.989503,-26.80,-14.35,-16.32,-16.39,-3.29,-11.021527,0.0
1,(주)CMG제약,58820,2006,2.50,6.22,232.13,0.00,-20.58,3.74,-35.59,...,-0.896302,0.296247,-14.082157,136.87,13.35,12.07,14.13,143.03,-0.677342,0.0
2,(주)CMG제약,58820,2010,2.71,7.41,306.73,8.48,11.86,23.74,-53.88,...,-0.896302,0.289649,-7.334934,198.14,-0.59,19.58,-1.60,148.77,0.950721,0.0
3,(주)CMG제약,58820,2014,1.95,5.27,325.47,2.65,11.96,37.07,1.96,...,-0.896302,0.155866,3.630886,-233.86,-1.39,-2.09,-0.68,-206.21,2.905378,0.0
4,(주)CMG제약,58820,2015,2.10,5.66,297.75,0.00,18.85,36.26,-9.51,...,-0.896302,0.171743,-18.504432,-27.72,-0.81,-11.47,-5.26,-33.02,-35.835295,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,잘만테크(주),90120,2014,1.56,3.24,50.12,56.44,-59.33,6.84,-53.39,...,-0.861568,0.952132,-6.782718,-79.98,-7.70,-57.55,-31.35,-72.53,-8.624891,1.0
60,제이앤유글로벌,86200,2015,3.69,7.07,159.68,16.90,30.06,29.86,-0.98,...,0.096090,0.534729,-0.361713,44.91,21.73,21.37,8.50,25.50,4.805396,1.0
61,평안물산(주),37240,2010,5.98,58.51,267.09,14.18,-49.19,-61.15,-171.19,...,-3.183320,0.197237,-18.565483,149.06,-44.72,-94.72,-9.98,149.43,-10.641540,1.0
62,퓨쳐비젼(주),42570,2007,2.52,58.51,14.61,57.72,-11.99,-5.22,-97.90,...,-3.394696,1.696991,-41.181406,-1932.08,-15.02,-44.05,-14.14,-1667.67,22.597350,1.0


In [35]:
expansion_x = expansion.iloc[:,3:-1]
expansion_y = expansion['부도']

In [36]:
expansion_x

,매출채권회전률,매입채무회전률,유동비율,차입금의존도,매출액증가율,매출액총이익률,매출액영업이익률,매출액순이익률,총자본영업이익률,자기자본순이익률,...,총자본회전률,RETA,TLTA,이자보상배율,vol유동비율,vol매출액총이익률,vol매출액영업이익률,vol총자본영업이익률,vol당좌비율,vol이자보상배율
0,18.91,58.88,114.21,53.16,66.81,0.00,-14.46,-8.54,-18.98,-29.43,...,1.45,-0.139228,0.575696,-9.989503,-26.80,-14.35,-16.32,-16.39,-3.29,-11.021527
1,2.50,6.22,232.13,0.00,-20.58,3.74,-35.59,-102.58,-12.93,-65.10,...,0.36,-0.896302,0.296247,-14.082157,136.87,13.35,12.07,14.13,143.03,-0.677342
2,2.71,7.41,306.73,8.48,11.86,23.74,-53.88,-67.39,-18.98,-57.61,...,0.48,-0.896302,0.289649,-7.334934,198.14,-0.59,19.58,-1.60,148.77,0.950721
3,1.95,5.27,325.47,2.65,11.96,37.07,1.96,0.75,0.78,0.35,...,0.40,-0.896302,0.155866,3.630886,-233.86,-1.39,-2.09,-0.68,-206.21,2.905378
4,2.10,5.66,297.75,0.00,18.85,36.26,-9.51,-13.49,-4.48,-7.61,...,0.47,-0.896302,0.171743,-18.504432,-27.72,-0.81,-11.47,-5.26,-33.02,-35.835295
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,1.56,3.24,50.12,56.44,-59.33,6.84,-53.39,-66.77,-27.10,-138.28,...,0.51,-0.861568,0.952132,-6.782718,-79.98,-7.70,-57.55,-31.35,-72.53,-8.624891
60,3.69,7.07,159.68,16.90,30.06,29.86,-0.98,-28.57,-0.50,-27.95,...,0.51,0.096090,0.534729,-0.361713,44.91,21.73,21.37,8.50,25.50,4.805396
61,5.98,58.51,267.09,14.18,-49.19,-61.15,-171.19,-194.40,-26.00,-38.66,...,0.15,-3.183320,0.197237,-18.565483,149.06,-44.72,-94.72,-9.98,149.43,-10.641540
62,2.52,58.51,14.61,57.72,-11.99,-5.22,-97.90,-1013.76,-25.89,0.00,...,0.26,-3.394696,1.696991,-41.181406,-1932.08,-15.02,-44.05,-14.14,-1667.67,22.597350


# 확장기 Scaling 및 oversampling(SMOTE)

In [37]:
transition_normal = pd.read_csv('../Data/WIN/transition_normal_win.csv',encoding='cp949')
transition_default = pd.read_csv('../Data/WIN/transition_default_win.csv',encoding='cp949')
transition = pd.concat([transition_normal,transition_default],axis=0)

transition_00 = transition[transition['회계년도']==2000]
transition_01 = transition[transition['회계년도']==2001]
transition_05 = transition[transition['회계년도']==2005]
transition_11 = transition[transition['회계년도']==2011]
transition_13 = transition[transition['회계년도']==2013]
transition_17 = transition[transition['회계년도']==2017]

In [38]:
y_transition_00 = transition_00['부도']
y_transition_01 = transition_01['부도']
y_transition_05 = transition_05['부도']
y_transition_11 = transition_11['부도']
y_transition_13 = transition_13['부도']
y_transition_17 = transition_17['부도']


In [39]:
expansion_final_lst=['RETA','매출액영업이익률','총자본영업이익률','매출액순이익률','이자보상배율','vol매출액영업이익률','vol매출액총이익률']
expansion_tf = expansion[expansion_final_lst]
expansion_tf

,RETA,매출액영업이익률,총자본영업이익률,매출액순이익률,이자보상배율,vol매출액영업이익률,vol매출액총이익률
0,-0.139228,-14.46,-18.98,-8.54,-9.989503,-16.32,-14.35
1,-0.896302,-35.59,-12.93,-102.58,-14.082157,12.07,13.35
2,-0.896302,-53.88,-18.98,-67.39,-7.334934,19.58,-0.59
3,-0.896302,1.96,0.78,0.75,3.630886,-2.09,-1.39
4,-0.896302,-9.51,-4.48,-13.49,-18.504432,-11.47,-0.81
...,...,...,...,...,...,...,...
59,-0.861568,-53.39,-27.10,-66.77,-6.782718,-57.55,-7.70
60,0.096090,-0.98,-0.50,-28.57,-0.361713,21.37,21.73
61,-3.183320,-171.19,-26.00,-194.40,-18.565483,-94.72,-44.72
62,-3.394696,-97.90,-25.89,-1013.76,-41.181406,-44.05,-15.02


In [40]:
expansion

,회사명,거래소코드,회계년도,매출채권회전률,매입채무회전률,유동비율,차입금의존도,매출액증가율,매출액총이익률,매출액영업이익률,...,RETA,TLTA,이자보상배율,vol유동비율,vol매출액총이익률,vol매출액영업이익률,vol총자본영업이익률,vol당좌비율,vol이자보상배율,부도
0,(주)CMG제약,58820,2002,18.91,58.88,114.21,53.16,66.81,0.00,-14.46,...,-0.139228,0.575696,-9.989503,-26.80,-14.35,-16.32,-16.39,-3.29,-11.021527,0.0
1,(주)CMG제약,58820,2006,2.50,6.22,232.13,0.00,-20.58,3.74,-35.59,...,-0.896302,0.296247,-14.082157,136.87,13.35,12.07,14.13,143.03,-0.677342,0.0
2,(주)CMG제약,58820,2010,2.71,7.41,306.73,8.48,11.86,23.74,-53.88,...,-0.896302,0.289649,-7.334934,198.14,-0.59,19.58,-1.60,148.77,0.950721,0.0
3,(주)CMG제약,58820,2014,1.95,5.27,325.47,2.65,11.96,37.07,1.96,...,-0.896302,0.155866,3.630886,-233.86,-1.39,-2.09,-0.68,-206.21,2.905378,0.0
4,(주)CMG제약,58820,2015,2.10,5.66,297.75,0.00,18.85,36.26,-9.51,...,-0.896302,0.171743,-18.504432,-27.72,-0.81,-11.47,-5.26,-33.02,-35.835295,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,잘만테크(주),90120,2014,1.56,3.24,50.12,56.44,-59.33,6.84,-53.39,...,-0.861568,0.952132,-6.782718,-79.98,-7.70,-57.55,-31.35,-72.53,-8.624891,1.0
60,제이앤유글로벌,86200,2015,3.69,7.07,159.68,16.90,30.06,29.86,-0.98,...,0.096090,0.534729,-0.361713,44.91,21.73,21.37,8.50,25.50,4.805396,1.0
61,평안물산(주),37240,2010,5.98,58.51,267.09,14.18,-49.19,-61.15,-171.19,...,-3.183320,0.197237,-18.565483,149.06,-44.72,-94.72,-9.98,149.43,-10.641540,1.0
62,퓨쳐비젼(주),42570,2007,2.52,58.51,14.61,57.72,-11.99,-5.22,-97.90,...,-3.394696,1.696991,-41.181406,-1932.08,-15.02,-44.05,-14.14,-1667.67,22.597350,1.0


In [41]:
expansion_x_tf = expansion[expansion_final_lst]
expansion_y_tf = expansion['부도']

transition_00_tf
transition_01_tf
transition_05_tf
transition_11_tf
transition_13_tf = scaler_std.transform(transition_13)
 
transition_17_tf

In [42]:
A=[transition_00,transition_01,transition_05,transition_11,transition_13,transition_17]
for i in range(len(A)):
    A[i] = A[i][expansion_final_lst]
    A[i] = A[i].reset_index(drop=True)

X_transition_00=A[0]
X_transition_01=A[1]
X_transition_05=A[2]
X_transition_11=A[3]
X_transition_13=A[4]
X_transition_17=A[5]

X_transition_00
y_transition_00

In [43]:
from sklearn.model_selection import train_test_split
X_train,RAW_X_test,y_train,RAW_y_test = train_test_split(expansion_x_tf,expansion_y_tf,test_size = 0.3, stratify = expansion_y, random_state=42)

In [44]:
X_transition_00.columns

Index(['RETA', '매출액영업이익률', '총자본영업이익률', '매출액순이익률', '이자보상배율', 'vol매출액영업이익률',
       'vol매출액총이익률'],
      dtype='object')

In [45]:
expansion_x_tf.columns

Index(['RETA', '매출액영업이익률', '총자본영업이익률', '매출액순이익률', '이자보상배율', 'vol매출액영업이익률',
       'vol매출액총이익률'],
      dtype='object')

In [46]:
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

scaler_std = StandardScaler()
sm = SMOTE(sampling_strategy={1:int(y_train.value_counts().iloc[0]/2),0:y_train.value_counts().iloc[0]},random_state=1024)

X_train_std = scaler_std.fit_transform(X_train)
X_train_std = pd.DataFrame(X_train_std)
X_resampled_std, y_resampled_std = sm.fit_resample(X_train_std,y_train)
X_resampled_std.columns = expansion_x_tf.columns

X_test_std = scaler_std.transform(RAW_X_test)
X_test_std=pd.DataFrame(X_test_std)
X_test_std.columns = expansion_x_tf.columns

X_transition_00 = scaler_std.transform(X_transition_00)
X_transition_00 = pd.DataFrame(X_transition_00)
X_transition_00.columns = expansion_x_tf.columns

X_transition_01 = scaler_std.transform(X_transition_01)
X_transition_01 = pd.DataFrame(X_transition_01)
X_transition_01.columns = expansion_x_tf.columns

X_transition_05 = scaler_std.transform(X_transition_05)
X_transition_05 = pd.DataFrame(X_transition_05)
X_transition_05.columns = expansion_x_tf.columns

X_transition_11 = scaler_std.transform(X_transition_11)
X_transition_11 = pd.DataFrame(X_transition_11)
X_transition_11.columns = expansion_x_tf.columns

X_transition_13 = scaler_std.transform(X_transition_13)
X_transition_13 = pd.DataFrame(X_transition_13)
X_transition_13.columns = expansion_x_tf.columns

X_transition_17 = scaler_std.transform(X_transition_17)
X_transition_17 = pd.DataFrame(X_transition_17)
X_transition_17.columns = expansion_x_tf.columns


print('After OverSampling, the shape of train_X: {}'.format(X_resampled_std.shape))
print('After OverSampling, the shape of train_y: {} \n'.format(y_resampled_std.shape))
print('StandardScaler - SMOTE 적용 후 값의 분포 :\n',y_resampled_std.value_counts())

RAW_y_test.reset_index(drop=True,inplace=True)

After OverSampling, the shape of train_X: (7534, 7)
After OverSampling, the shape of train_y: (7534,) 

StandardScaler - SMOTE 적용 후 값의 분포 :
 0.0    5023
1.0    2511
Name: 부도, dtype: int64


In [47]:
# test_std = pd.concat([X_test_std,RAW_y_test],axis=1)
# test_std.to_csv('../Data/WIN_UP/recession_win_std_test.csv',index=False,encoding='cp949')

# upsample_std = pd.concat([X_resampled_std,y_resampled_std],axis=1)
# upsample_std.to_csv('../Data/WIN_UP/recession_win_std_smote.csv',index=False, encoding='cp949')

In [48]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수 ()
      def grid(self,model_name, X_train=X_resampled_std, y_train=y_resampled_std, X_test=X_transition_00,y_test=y_transition_00):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df
        
list = ['XGBClassifier','RandomForestClassifier']
model = Model_Optimization()
model.get_model_socre(list)
model_df.to_csv('../Data/result/2000년_확장.csv',index=False, encoding='cp949')
model_df


  0%|          | 0/2 [00:00<?, ?it/s]

[19:57:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:57:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델명: XGBClassifier
학습 데이터 최적 파라미터
: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
학습 데이터 최고 정확도
: 0.997
     Negative(0)  Positive(1)  y_pred
0       0.999998     0.000002     0.0
1       0.998670     0.001330     0.0
2       0.998249     0.001751     0.0
3   

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.8,0.8,XGBClassifier,0.985,0.25,0.5,0.333333,0.881657
0.8,0.8,RandomForestClassifier,0.994,1.00,0.5,0.666667,0.997041


In [49]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수 ()
      def grid(self,model_name, X_train=X_resampled_std, y_train=y_resampled_std, X_test=X_transition_01,y_test=y_transition_01):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df
        
list = ['XGBClassifier','RandomForestClassifier']
model = Model_Optimization()
model.get_model_socre(list)
model_df.to_csv('../Data/result/2001년_확장.csv',index=False, encoding='cp949')

model_df


  0%|          | 0/2 [00:00<?, ?it/s]

[19:58:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:58:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델명: XGBClassifier
학습 데이터 최적 파라미터
: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
학습 데이터 최고 정확도
: 0.997
     Negative(0)  Positive(1)  y_pred
0       0.999945     0.000055     0.0
1       0.999928     0.000072     0.0
2       0.999998     0.000002     0.0
3   

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.3,0.3,XGBClassifier,0.968,0.038462,0.166667,0.0625,0.782738
0.6,0.6,RandomForestClassifier,0.981,0.100000,0.166667,0.1250,0.758135


In [50]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수 ()
      def grid(self,model_name, X_train=X_resampled_std, y_train=y_resampled_std, X_test=X_transition_05,y_test=y_transition_05):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df
        
list = ['XGBClassifier','RandomForestClassifier']
model = Model_Optimization()
model.get_model_socre(list)
model_df.to_csv('../Data/result/2005년_확장.csv',index=False, encoding='cp949')

model_df


  0%|          | 0/2 [00:00<?, ?it/s]

[19:58:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:58:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델명: XGBClassifier
학습 데이터 최적 파라미터
: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
학습 데이터 최고 정확도
: 0.997
     Negative(0)  Positive(1)  y_pred
0       0.479603     0.520397     1.0
1       0.986652     0.013347     0.0
2       0.999998     0.000002     0.0
3   

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.9,0.9,XGBClassifier,0.940,0.030303,0.5,0.057143,0.929009
0.8,0.8,RandomForestClassifier,0.959,1.000000,0.5,0.666667,0.921348


In [51]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수 ()
      def grid(self,model_name, X_train=X_resampled_std, y_train=y_resampled_std, X_test=X_transition_11,y_test=y_transition_11):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df
        
list = ['XGBClassifier','RandomForestClassifier']
model = Model_Optimization()
model.get_model_socre(list)
model_df.to_csv('../Data/result/2011년_확장.csv',index=False, encoding='cp949')

model_df


  0%|          | 0/2 [00:00<?, ?it/s]

[19:59:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:59:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델명: XGBClassifier
학습 데이터 최적 파라미터
: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
학습 데이터 최고 정확도
: 0.997
       Negative(0)   Positive(1)  y_pred
0     9.982249e-01  1.775139e-03     0.0
1     9.991938e-01  8.062070e-04     0.0
2     9.999996e-01  4.048274e-07 

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.9,0.9,XGBClassifier,0.976,0.529412,0.6,0.5625,0.92611
0.8,0.8,RandomForestClassifier,0.982,1.000000,0.6,0.7500,0.94662


In [52]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수 ()
      def grid(self,model_name, X_train=X_resampled_std, y_train=y_resampled_std, X_test=X_transition_13,y_test=y_transition_13):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df
        
list = ['XGBClassifier','RandomForestClassifier']
model = Model_Optimization()
model.get_model_socre(list)
model_df.to_csv('../Data/result/2013년_확장.csv',index=False, encoding='cp949')

model_df


  0%|          | 0/2 [00:00<?, ?it/s]

[19:59:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[19:59:32] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델명: XGBClassifier
학습 데이터 최적 파라미터
: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
학습 데이터 최고 정확도
: 0.997
      Negative(0)   Positive(1)  y_pred
0        0.999963  3.725031e-05     0.0
1        0.999675  3.252227e-04     0.0
2        0.999999  1.145775e-06     

,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.6,0.6,XGBClassifier,0.978,0.346154,0.818182,0.486486,0.905310
0.8,0.8,RandomForestClassifier,0.981,1.000000,0.545455,0.705882,0.911904


In [53]:
from sklearn.metrics import roc_curve, roc_auc_score, precision_recall_curve, accuracy_score, f1_score
from sklearn.svm import SVC
from sklearn.linear_model import LinearRegression,  LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from tqdm.notebook import tqdm
from sklearn.preprocessing import Binarizer
from sklearn.metrics import recall_score, precision_score
from sklearn.model_selection import GridSearchCV

# df = pd.DataFrame(columns =['models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score'])



class Model_Optimization():

      def __init__(self):
            
            global model_df
            model_df = pd.DataFrame(columns =['thresholds','models','Accuracy','Precision', 'Recall', 'F1 score','Auc_Score']) 


      #최적 파라미터 찾는 함수 ()
      def grid(self,model_name, X_train=X_resampled_std, y_train=y_resampled_std, X_test=X_transition_17,y_test=y_transition_17):
            global model_df

            self.model_name = model_name
            self.X_test = X_test
            self.y_test = y_test
            self.X_train = X_train
            self.y_train  = y_train

            if model_name == "DecisionTreeClassifier":
                  dt = DecisionTreeClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "RandomForestClassifier":
                  dt = RandomForestClassifier()
                  parameters = [{"max_depth": [3,5,7], "min_samples_split":[3,5,7]}]
                  
            elif model_name == "LogisticRegression":
                  dt = LogisticRegression()
                  parameters = {'penalty':['l2', 'l1'],
                  'C':[0.01, 0.1, 1, 5, 10]}
                  
            elif model_name == "Linear SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'kernel':['linear'], 'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]
                  
            elif model_name == "RBF SVM": 
                  dt = SVC(probability=True)
                  parameters = [{'gamma':[0.001, 0.01, 0.1, 1, 10, 100],'kernel':['rbf'],'C':[0.001, 0.01, 0.025, 0.1, 1, 10, 100]}]

            elif model_name == "AdaBoostClassifier":
                  dt= AdaBoostClassifier(random_state=0)
                  parameters={'n_estimators':[5]}
                  
            elif model_name == "GradientBoostingClassifier":
                  dt=GradientBoostingClassifier(random_state=0)
                  parameters = {
                  'n_estimators' : [100, 500],
                  'learning_rate' : [0.05, 0.1]
                              }

            elif model_name == "XGBClassifier":
                  dt=XGBClassifier(random_state=0,verbose=1)
                  parameters={'n_estimators':[100, 500], 'learning_rate':[0.05, 0.1], 'max_depth':[3,4]}
                  
            # elif model_name == "LGBMClassifier":
            #       dt=LGBMClassifier(random_state=0)
            #       parameters={'n_estimators':[400,800], 'learning_rate':[0.05, 0.1] , 'max_depth':[3,4]}
            
            elif model_name == "KNeighborsClassifier":
                  dt = KNeighborsClassifier()
                  parameters = {'n_neighbors':[3]}
                  
            elif model_name == 'MLPClassifier':
                  dt = MLPClassifier(random_state=0)
                  parameters={'max_iter':[1000], 'hidden_layer_sizes':[1], 'activation':['logistic'],
                              'solver':['sgd'], 'alpha':[0.01], 'batch_size':[32],
                              'learning_rate_init':[0.1], 'max_iter':[500]}
                  
            elif model_name == 'GaussianProcessClassifier':
                  dt = GaussianProcessClassifier(random_state=0)
                  parameters={'kernel': [1.0*RBF(1.0)]}
                  
            elif model_name == 'GaussianNB':
                  dt = GaussianNB()
                  parameters={}
                  
            elif model_name =='QuadraticDiscriminantAnalysis':
                  #선형판별분석
                  dt = QuadraticDiscriminantAnalysis()
                  parameters={}
                  
            # 최적 파라미터 찾기 
            self.grid_dt  = GridSearchCV(dt, param_grid = parameters, cv=5, refit =True, n_jobs=-1)
            self.grid_dt.fit(self.X_train, self.y_train)
            
            print(f"모델명: {model_name}")
            print(f"학습 데이터 최적 파라미터\n: {self.grid_dt.best_params_}")
            print(f"학습 데이터 최고 정확도\n: {self.grid_dt.best_score_:.3f}")

            #지도학습 알고리즘 
            estimator = self.grid_dt.best_estimator_
            y_pred = pd.DataFrame(estimator.predict(self.X_test))
            y_pred_probability = pd.DataFrame(estimator.predict_proba(self.X_test))
            # print(f"예측 정확도\n: {accuracy_score(self.y_test, y_pred):.3f}")

            prediction = pd.concat([y_pred_probability, y_pred], axis = 1)

            prediction.columns = ["Negative(0)", "Positive(1)", "y_pred"]

            print(prediction)
            self.pred_proba_1 = np.array(prediction["Positive(1)"]).reshape(-1, 1)
            self.accuracy= round(accuracy_score(self.y_test, y_pred),3)
            self.FPRs, self.TPRs, self.thresholds = roc_curve(self.y_test, self.pred_proba_1)
            
            return estimator



      # 단일 모델 임계치별 score 출력 하는 함수 *주의  self.thresholds 값 내부 리스트 수정해줘야함 
      def get_thresholds_score(self):
            global model_df

            
            self.thresholds = [0.1, 0.2 , 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]

            #평가지표용 리스트생성
            precisions = []
            recalls = []
            f1_scores = []
            auc_scores=[]
            threshold_ =[]
            
            
            
            for threshold in self.thresholds:
                  binarizer = Binarizer(threshold= threshold)
                  # print(threshold)
                  # 임계점 지정하여 Binariazer() 객체 생성
                  pred_proba = binarizer.fit_transform(self.pred_proba_1)
                  # print(pred_proba)
                  # 임계점을 기준으로 데이터 변환

                  precision = precision_score(self.y_test, pred_proba)
                  recall = recall_score(self.y_test, pred_proba)
                  f1score = f1_score(self.y_test, pred_proba)
                  auc_score=roc_auc_score(self.y_test, self.pred_proba_1)

                  precisions.append(precision)
                  recalls.append(recall)
                  f1_scores.append(f1score)
                  auc_scores.append(auc_score)
                  threshold_.append(threshold)
                  

            results = pd.DataFrame(data = {"thresholds":threshold_, "models":self.model_name,"Accuracy":self.accuracy,"Precision": precisions,
                                          "Recall": recalls,
                                          "F1 score": f1_scores,"Auc_Score":auc_scores}, index = self.thresholds)                 

            print(f'단일 모델 results : {results}')      
            return results




      #모든 모델 성능 비교 함수 
      def get_model_socre(self,input_list):
            global model_df 

            mod = Model_Optimization()
            for i in tqdm(input_list):
                  mod.grid(model_name=i)
                  results_df = mod.get_thresholds_score()

                  #평가지표 데이터 프레임화 
                  print(results_df)
                  #기준 평가지표로 정렬
                  results_df.sort_values("F1 score", ascending=False, inplace=True)
                  new_model_df= results_df.iloc[:1]
                  model_df = pd.concat([model_df,new_model_df])
                  
            return model_df
        
list = ['XGBClassifier','RandomForestClassifier']
model = Model_Optimization()
model.get_model_socre(list)
model_df.to_csv('../Data/result/2017년_확장.csv',index=False, encoding='cp949')

model_df


  0%|          | 0/2 [00:00<?, ?it/s]

[20:00:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "verbose" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[20:00:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
모델명: XGBClassifier
학습 데이터 최적 파라미터
: {'learning_rate': 0.1, 'max_depth': 3, 'n_estimators': 500}
학습 데이터 최고 정확도
: 0.997
      Negative(0)  Positive(1)  y_pred
0        0.999997     0.000003     0.0
1        0.999998     0.000002     0.0
2        0.130568     0.869432     1.0


,thresholds,models,Accuracy,Precision,Recall,F1 score,Auc_Score
0.1,0.1,XGBClassifier,0.974,0.0,0.0,0.0,0.348004
0.1,0.1,RandomForestClassifier,0.979,0.0,0.0,0.0,0.372051
